# 02. Forest Connectivity Analysis

This is the **main analytical workflow**. We will:
1. Load LULC data (from API or sample).
2. Extract Forest Mask.
3. Compute Distance from Edge.
4. Classify Connectivity (Core, Edge, Fragmented).
5. Vectorize Results.
6. Visualize and Export.


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
from src.connectivity import ConnectivityAnalyzer
from src.vectorization import raster_to_polygons, export_results
from src.visualization import plot_connectivity_map, plot_statistics
import os

# Ensure output dir exists
os.makedirs("outputs/vectors", exist_ok=True)
os.makedirs("outputs/maps", exist_ok=True)

## 1. Load Data

In [ ]:
# For this demo, we use the local sample to ensure reproducibility in the notebook
# In production, replace this with client.fetch_lulc_raster(...)
from utils.sample_data import generate_sample_lulc

sample_path = "data/sample_lulc.tif"
if not os.path.exists(sample_path):
    generate_sample_lulc(sample_path)

with rasterio.open(sample_path) as src:
    lulc_array = src.read(1)
    transform = src.transform
    crs = src.crs
    
print(f"Loaded LULC: {lulc_array.shape}, CRS: {crs}")

## 2. Initialize Analyzer & Extract Mask
We define Forest as Class 3 (Deciduous) and 4 (Evergreen).

In [ ]:
analyzer = ConnectivityAnalyzer(resolution=30)
forest_mask = analyzer.extract_forest_mask(lulc_array, forest_classes=[3, 4])

plt.figure(figsize=(6, 6))
plt.imshow(forest_mask, cmap='Greens')
plt.title("Forest Mask")
plt.axis('off')
plt.show()

## 3. Compute Connectivity
Calculate distance from edge and classify zones.

In [ ]:
dists = analyzer.compute_distance_from_edge(forest_mask)
connectivity = analyzer.classify_connectivity(dists)

# Calculate Stats
stats = analyzer.calculate_statistics(connectivity)
print("Connectivity Statistics:")
for k, v in stats.items():
    print(f"  {k}: {v:.2f}")

## 4. Visualize Results

In [ ]:
fig_map = plot_connectivity_map(connectivity, transform)
plt.show()

fig_stats = plot_statistics(stats)
plt.show()

# Save
fig_map.savefig("outputs/maps/connectivity_map.png")

## 5. Vectorize & Export
Convert to polygons for GIS use.

In [ ]:
gdf = raster_to_polygons(connectivity, transform, crs=str(crs))
print(f"Generated {len(gdf)} polygons.")

if not gdf.empty:
    output_file = "outputs/vectors/forest_connectivity.geojson"
    export_results(gdf, output_file)
    print(f"Exported to {output_file}")
    print(gdf.head())